In [3]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [4]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ... ENTER YOUR CODE HERE ...
    outvec = np.zeros(len(vec))
    outvec[0] = np.linalg.norm(vec)
    
    x_y_diff = (vec - outvec)
    x_y_diff[0] = -np.sum([i*i for i in vec[1:]]) / (vec[0] + outvec[0])
    
    u = x_y_diff / np.linalg.norm(x_y_diff)
    u_perpendicular =  np.multiply(u[np.newaxis].T, u[np.newaxis])
    H = np.eye(len(vec)) - 2 * u_perpendicular
    return outvec, H

Test your function using tests below:

In [19]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol = 3e-16)

In [20]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [61]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    q = np.eye(m)
    # ... ENTER YOUR CODE HERE ...
    for i in range(n):
        _, h = householder(a1[i:, i])
        H = np.eye(m)     
        H[i:, i:] = h
        q = np.dot(q, H)
        a1 = np.dot(H, a1)
    return q, a1

In [62]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [63]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))

q, r = qr_decomp(a)
# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [64]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

q and r do not agree with qq and rr because in this case the QR decomposition is not unique. As the matrix a is not full rank.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [158]:
def householderQ(vec):
    """
    A new Function to return reflection vectors
    """
    outvec = np.zeros(len(vec))
    outvec[0] = np.linalg.norm(vec)
        
    x_y_diff = (vec - outvec)
    x_y_diff[0] = -np.sum([i*i for i in vec[1:]]) / (vec[0] + outvec[0])
    u = x_y_diff / np.linalg.norm(x_y_diff)
    return outvec, u

In [192]:
def get_Q_from_RefVectors(outvecs):
    """
    ref vectors 
    """
    m = len(outvecs[0])
    q = np.eye(m)
    for vec in outvecs:
        print(vec)
        q = (np.eye(m) + (-2)* (vec[np.newaxis].T)@vec[np.newaxis])@q
    return q.T

In [199]:
def qr_decomp_without_explicit_householder(a):
    """
    qr decomp without any explicit householder
    """
    a1 = np.array(a, copy=True, dtype=float)

    m, n = a1.shape
    q = np.eye(m)
    outputs = []
    # ... ENTER YOUR CODE HERE ...
    for i in range(n):
        vec = a[i:, i]
        outvec, u = householderQ(vec)
        a_temp = a1[i:,i:]
        
        U = np.zeros(m)
        U[i:]=u
        a1 = a1 - 2 * np.dot(U[np.newaxis].T,  np.dot(U[np.newaxis], a1))

        outputs.append(U)
    return outputs, a1

In [200]:
vecs, r = qr_decomp_without_explicit_householder(a)

In [201]:
Q,R = qr_decomp(a)

In [202]:
R-r

array([[ 0.        ,  0.        ,  0.        ],
       [-0.        ,  0.08668256,  0.09782505],
       [ 0.        ,  0.05978056,  0.81398391],
       [-0.        , -0.21409591, -0.59610183],
       [ 0.        , -0.29828915,  0.03435307]])

In [203]:
Q1 = get_Q_from_RefVectors(vecs)

[-0.6570196   0.42644006  0.15011669  0.47562065  0.37111197]
[ 0.         -0.46489311  0.62776504  0.28012591  0.55795603]
[ 0.          0.         -0.28427536  0.77103476  0.56981832]


In [204]:
# To check that Q1 is similar to Q
Q1-Q

array([[ 0.        ,  0.31303539,  0.1757079 , -1.06342255, -0.71380559],
       [ 0.        , -0.07686768,  1.2516353 , -0.0769656 ,  1.03424963],
       [ 0.        , -0.26979352, -0.82296582,  1.16583527,  0.47144398],
       [ 0.        ,  0.15000873, -0.6073748 , -0.11074883, -0.71633852],
       [ 0.        , -0.08251062, -0.37616372,  0.05678206, -0.26105627]])

In [205]:
assert_allclose(np.dot(Q1, r), a)

In [206]:
np.dot(Q1, r)

array([[0.19151945, 0.62210877, 0.43772774],
       [0.78535858, 0.77997581, 0.27259261],
       [0.27646426, 0.80187218, 0.95813935],
       [0.87593263, 0.35781727, 0.50099513],
       [0.68346294, 0.71270203, 0.37025075]])

In [207]:
a

array([[0.19151945, 0.62210877, 0.43772774],
       [0.78535858, 0.77997581, 0.27259261],
       [0.27646426, 0.80187218, 0.95813935],
       [0.87593263, 0.35781727, 0.50099513],
       [0.68346294, 0.71270203, 0.37025075]])

this looks like even if Q and R do not match;the final QR matrix is similar